# Creation of BlackBox Models for the Adult dataset

In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# ADD OTHER FOLDERS TO THIS LIST TO ADD THEM TO THE sys.path
modules_to_add = [""]

this_file = os.path.abspath('')

for module in modules_to_add:
    p = Path(this_file).parent / module 
    if p.exists():
        sys.path.append(str(p))
        print(f"ADDED: {p}")
    else:
        print(f"ERROR: {p} doesn't exist")

ADDED: /home/gerardozinno/Desktop/Tesi/Code/mlem


In [3]:
print(sys.path)

['/home/gerardozinno/Desktop/Tesi/Code/mlem/notebooks', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python39.zip', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9/lib-dynload', '', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/IPython/extensions', '/home/gerardozinno/.ipython', '/home/gerardozinno/Desktop/Tesi/Code/mlem']


In [13]:
DATASET_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
WHOLE_DATASET_PATH = Path("../data/adult/adult.csv")
TRAIN_PATH = WHOLE_DATASET_PATH.parent / "train" / "train.csv"
TEST_PATH  = WHOLE_DATASET_PATH.parent / "test" / "test.csv"

In [ ]:
TRAIN_SPLIT = .8
RAND_SEED   = 1234

## Dataset creation and cleaning

In [38]:
if not WHOLE_DATASET_PATH.exists():
    print(f"downloading dataset from {DATASET_URL}")
    columns = ['Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-num', 'Marital-status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital-gain', 'Capital-loss', 'Hours-per-week', 'Native-country', 'Target']
    df = pd.read_csv(DATASET_URL, names=columns)
    print("dataset downloaded")
    print("Cleaning and preprocessing dataset:")
    print("\tdropping duplicates")
    df.drop_duplicates(inplace=True)
    print("\ttrimming strings")
    df_str = df.select_dtypes(['object'])
    df[df_str.columns] = df_str.apply(lambda x: x.str.strip())
    
    WHOLE_DATASET_PATH.parent.mkdir(exist_ok=True)
    df.to_csv(WHOLE_DATASET_PATH)

    
if not (TRAIN_PATH.exists() and TEST_PATH.exists()):
    print(f"Couldn't find the train and/or test dataset(s) in:\n\t{TRAIN_PATH}\n\t{TEST_PATH}\n")
    if not WHOLE_DATASET_PATH.exists():
        print(f"ERROR: Couldn't even find {WHOLE_DATASET_PATH}")
        raise Exception("Can't find dataset")
    else:
        
        print(f"Creating train and test sets with a split of {TRAIN_SPLIT}% - {1-TRAIN_SPLIT:.2f}% and {RAND_SEED} as random seed")
        print('The dataset is split "as is", without preprocessing. The selection of the right columns is made by the respective Dataloader')
        df = pd.read_csv(WHOLE_DATASET_PATH)
        train, test = train_test_split(df, train_size=TRAIN_SPLIT, shuffle=True, random_state=RAND_SEED)
        TRAIN_PATH.parent.mkdir(exist_ok=True)
        TEST_PATH.parent.mkdir(exist_ok=True)
        train.to_csv(TRAIN_PATH, index=False)
        test.to_csv(TEST_PATH, index=False)
        print("train and test datasets created")